## Imports

In [1]:
from frontier_accuracy import fill_grid, get_connection_points
import matplotlib.pyplot as plt
from frontier_algorithms.convolutional import ConvolutionalFrontierDetector
from frontier_algorithms.simple_search import SimpleFrontierDetector
import numpy as np
from occupancy_grid import OccupancyGrid
from ConcaveHull import ConcaveHull

## Gather Ground Truth

For this section, the 'Simple Frontier Detector' is set to just return all frontiers, before performing clustering. This ensures every possible frontier is found, and then is used as the baseline.

In [ ]:
map_folder = "maps/"
map_extension = ".tmj"
maps = [("extra-medium", 350, 350), ("large-field-large-explored", 500, 500), ("large-field-medium-explored", 500, 500), ("medium-field-medium-explored", 250, 250), ("medium-field-large-explored", 250, 250), ("star", 500, 500)]

In [ ]:
ground_truth_detector = SimpleFrontierDetector()
alpha_values = [1000, 150, 120, 140, 100]

current_map, x, y = maps[0]
occupancy_grid = OccupancyGrid(f"{map_folder}{current_map}{map_extension}", True)
frontiers = ground_truth_detector.identify_frontiers(occupancy_grid.as_flat(), occupancy_grid.width, occupancy_grid.height, np.array([[250, 250]]), False)

In [ ]:
world_map = np.zeros((occupancy_grid.width, occupancy_grid.height))
ground_truth_map = np.zeros((occupancy_grid.height, occupancy_grid.width))
fill_grid(ground_truth_map, frontiers, x, y)
plt.imshow(ground_truth_map, 'gray')

In [ ]:
np.unique(ground_truth_map)

## Load ground truth

In [ ]:
ground_truth = np.load('med-field-med-explored-ground-truth.npy')

occupancy_grid = OccupancyGrid('maps/medium-field-medium-explored.tmj', True)

frontier_detector = ConvolutionalFrontierDetector(64)

In [ ]:
frontiers = frontier_detector(occupancy_grid.as_flat(), occupancy_grid.width, occupancy_grid.height, np.array([[250, 250]]))

In [ ]:
x, y = get_connection_points(frontiers, 250)

# true_x, true_y = [point[0] for point in frontiers], [point[1] for point in frontiers]
# plt.plot(true_x, true_y, 'ro')

plt.plot(x, y, 'b.-')
plt.show()

In [ ]:
world_map = np.zeros((occupancy_grid.width, occupancy_grid.height))

ground_truth_map = np.zeros_like(world_map)

fill_grid(ground_truth_map, ground_truth, 250, 250)

fill_grid(world_map, frontiers, 250, 250, alpha=250)

In [ ]:
plt.imshow(world_map, 'gray')

In [ ]:
plt.imshow(ground_truth_map, 'gray')

In [ ]:
difference = np.abs(ground_truth_map - world_map)
print(np.sum(difference))
plt.imshow(difference, 'gray')

percent_off = np.sum(difference) / np.sum(ground_truth_map)

print(percent_off)